In [1]:
# from dolfinx import mesh,fem,plot as msh1,fem,plot
import dolfinx.mesh as mesh
import dolfinx.fem as fem
import dolfinx.plot as plot
import ufl
import mpi4py.MPI as mpi
# import petsc4py.PETSc as pet
import numpy as np

import pyvista

In [2]:
N = 20
domain = mesh.create_unit_square(mpi.COMM_WORLD, N, N)
space = fem.FunctionSpace(domain, ('CG', 1))

In [3]:
# Dot = ufl.dot
Dot = ufl.inner
Grad = ufl.grad
dx = ufl.dx
ds = ufl.ds
Div = ufl.div

In [4]:
u0 = fem.Function(space)
x = ufl.SpatialCoordinate(domain)
u_expression = 1 + x[0]**2 + x[1]**2
u0.interpolate(fem.Expression(u_expression, space.element.interpolation_points))
f = -Div(Grad(u_expression))

In [5]:
u = ufl.TrialFunction(space)
v = ufl.TestFunction(space)

n = ufl.FacetNormal(domain)
h = 2 * ufl.Circumradius(domain)
alfa = fem.Constant(domain, np.float64(10))
a = Dot(Grad(u), Grad(v)) * dx -\
    Dot(n, Grad(u)) * v * ds - Dot(n, Grad(v)) * u * ds +\
    alfa / h * u * v * ds
L = Dot(f, v) * dx -\
    Dot(n, Grad(v)) * u0 * ds + alfa/h * u0 * v* ds


In [6]:
problem= fem.petsc.LinearProblem(a,L)
uh=problem.solve()

In [7]:
error_form = fem.form(Dot(uh - u0, uh - u0) * dx)
error_local = fem.assemble_scalar(error_form)
errorL2 = np.sqrt(domain.comm.allreduce(error_local, op=mpi.SUM))
if domain.comm.rank == 0:
    print(fr"L^2-error: {errorL2:.2e}")

L^2-error: 1.05e-04


In [17]:
pyvista.set_jupyter_backend("pythreejs")

grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(space))
grid.point_data["u"] = uh.x.array.real
grid.set_active_scalars("u")
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=False, show_scalar_bar=True)
plotter.view_xy()
pyvista.OFF_SCREEN = False
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("nitsche.png")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…